In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime

Домашняя работа
Наборы данных вида Transactions (несколько транзакций на одного клиента) трансформировать в таблицу, где cl_id будут уникальными (соответственно 4000 строк в train и 1000 строк в test
Для каждого cl_id будет уникальное целевое событие target_flag, а также уникальный канал привлечения клиента channel_type (клиент привлекается лишь однажды и с самого начала его записи присваивается значение канала привлечения)
При агрегации (pandas.DataFrame.groupby) по cl_id (или по связке cl_id, channel_type, target_flag) необходимо создавать производные фичи, идеи для таких фичей могут быть следующими:

общая сумма транзакций по каждой из trx_category
общая сумма транзакции по основным вылютам (напр. выделить рубли, доллары и евро - предположительно, это будут самые крупные категории)
общая сумма транзакций по категориям MCC кодов (например, выбрать основные/популярные MCC коды). ВНИМАНИ! Некоторые MCC коды из train могут быть не представлены в test. Про MCC коды в целом: http://www.banki.ru/wikibank/mcc-kod/; Справочник MCC кодов: https://mcc-codes.ru/code; Про некоторые категории кэшбека Росбанка: https://mcc-codes.ru/card/rosbank-sverkh-plus;
возможные агрегации по времени суток и дням недели - траты в выходные (праздники) или будни, в ночное время или в рабочее и т.д.

Обязательная часть: провести первичный анализ данных - посмотреть распределения признаков, выделить самые популярные MCC, помотреть активность клиентов по дням недели/времени, какие категории транзакции (trx_category) наиболее популярны и т.д. Получить инсайты, которые в дальнейшем помогут вам правильно подготовить фичи

In [3]:
raw_df = pd.read_csv('rosbank_train.csv')
cl_ids_test = np.random.choice(raw_df.cl_id.unique(), size=1000, replace=False)
cl_ids_test_set = set(cl_ids_test)
# create transactions dataset for train
transactions_train = raw_df[~raw_df.cl_id.isin(cl_ids_test)].copy()
print("Total transactions in train dataset: ", len(transactions_train))
# create transactions dataset for test
transactions_test = raw_df[raw_df.cl_id.isin(cl_ids_test)].copy()
print("Total transactions in test dataset: ", len(transactions_test))

Total transactions in train dataset:  391496
Total transactions in test dataset:  99017


In [4]:
transactions_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 391496 entries, 0 to 490512
Data columns (total 10 columns):
PERIOD          391496 non-null object
cl_id           391496 non-null int64
MCC             391496 non-null int64
channel_type    388861 non-null object
currency        391496 non-null int64
TRDATETIME      391496 non-null object
amount          391496 non-null float64
trx_category    391496 non-null object
target_flag     391496 non-null int64
target_sum      391496 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 32.9+ MB


In [55]:
transactions_train.head(9)

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0,0.0
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0,0.0
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0,0.0
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0,0.0
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0,0.0
5,01/10/2017,1,5814,NaN,810,16OCT17:00:00:00,380.0,POS,0,0.0
6,01/10/2017,1,5814,NaN,810,10OCT17:00:00:00,378.0,POS,0,0.0
7,01/10/2017,1,5814,NaN,810,16OCT17:00:00:00,199.0,POS,0,0.0
8,01/10/2017,1,5814,NaN,810,11OCT17:00:00:00,400.0,POS,0,0.0


In [110]:
#общая сумма транзакции по основным вылютам

first_iter = pd.DataFrame(raw_df[
    ['cl_id','currency','amount']
    ].groupby(['cl_id','currency']).agg(['sum']))

third_iter = pd.DataFrame(raw_df[
    ['cl_id','channel_type','target_flag']
    ].groupby(['cl_id']).agg(['max']))

second_iter = first_iter.join(third_iter, on = 'cl_id')
second_iter.head(5)

amount channel_type target_flag
                      sum          max         max
cl_id currency                                    
0     810        64383.00          NaN           0
1     810       266693.13          NaN           0
      978          884.91          NaN           0
5     504         1442.67          NaN           1
      810       544874.63          NaN           1

In [128]:
#Посмотреть распределения признаков
currency_split = pd.DataFrame(raw_df[['currency','amount']].groupby(['currency']).agg(['count']))
currency_split.head(5)

,amount
,count
currency,
32,12
36,17
44,2
51,60
124,41


In [134]:
currency_split.sort_values(by = ['amount','count'], ascending=False)

ValueError: The column label 'amount' is not unique.
For a multi-index, the label must be a tuple with elements corresponding to each level.